In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import logging

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
# ----------------- PARÁMETROS DEL DATASET -----------------
# Define la ruta a tu carpeta con las imágenes
data_dir = 'assets_num/' 
# Define el tamaño al que quieres redimensionar las imágenes
img_height = 28
img_width = 28
batch_size = 32

# Carga los datos desde el directorio sin redimensionar de forma automática
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width), # <-- Pasa el tamaño aquí
    batch_size=batch_size,
    color_mode='grayscale'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width), # <-- Pasa el tamaño aquí
    batch_size=batch_size,
    color_mode='grayscale'
)

class_names = train_dataset.class_names
print("Clases encontradas:", class_names)
num_classes = len(class_names)

# Normalización: ahora solo necesitamos escalar los valores
def normalize_image(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

# Aplica la normalización a ambos conjuntos de datos
train_dataset = train_dataset.map(normalize_image)
test_dataset = test_dataset.map(normalize_image)

# Cache y prefetch para mejorar el rendimiento
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
    predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img[...,0], cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("Prediccion: {}".format(class_names[predicted_label]), color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(num_classes), predictions_array, color="#888888")
    plt.ylim([0,1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(img_height, img_width, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(), # <-- Normalización por lotes
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.BatchNormalization(), # <-- Normalización por lotes
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5), # <-- Dropout (ej. 50%)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
    # ----------------- COMPILACIÓN Y ENTRENAMIENTO -----------------
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )



In [ ]:
import datetime
log_dir = "logs/num/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(
        train_dataset, 
        epochs=100, 
        validation_data=test_dataset,
        callbacks=[tensorboard_callback]
    )

    # Evaluar el modelo
test_loss, test_accuracy = model.evaluate(test_dataset)
print("Resultado en las pruebas: ", test_accuracy)
for test_images, test_labels in test_dataset.take(1):
    test_images = test_images.numpy()
    test_labels = test_labels.numpy()
    predictions = model.predict(test_images)
    
model.save('numeros2.keras') 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'logs/num/'


In [ ]:
!del /q "C:\Users\shado\AppData\Local\Temp\.tensorboard-info\*"

In [ ]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

In [ ]:
notebook.display(port=6006, height=512)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs'